In [ ]:
import re
from typing import List

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import optuna
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import keras

tqdm.pandas()

In [3]:
df = pd.read_csv("./data/train_folds.zip")
df_test = pd.read_csv("./data/Test.zip")

df1 = pd.read_csv("./data/train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("./data/train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("./data/train_pred_3.csv")
df3.columns = ["id", "pred_3"]
df4 = pd.read_csv("./data/train_pred_4.csv")
df4.columns = ["id", "pred_4"]
df5 = pd.read_csv("./data/train_pred_5.csv")
df5.columns = ["id", "pred_5"]
df6 = pd.read_csv("./data/train_pred_6.csv")
df6.columns = ["id", "pred_6"]

df_test1 = pd.read_csv("./data/test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("./data/test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("./data/test_pred_3.csv")
df_test3.columns = ["id", "pred_3"]
df_test4 = pd.read_csv("./data/test_pred_4.csv")
df_test4.columns = ["id", "pred_4"]
df_test5 = pd.read_csv("./data/test_pred_5.csv")
df_test5.columns = ["id", "pred_5"]
df_test6 = pd.read_csv("./data/test_pred_6.csv")
df_test6.columns = ["id", "pred_6"]

df = df.merge(df1, left_on="user_id", right_on='id', how="left")
df = df.merge(df2, left_on="user_id", right_on='id', how="left")
df = df.merge(df3, left_on="user_id", right_on='id', how="left")
df = df.merge(df4, left_on="user_id", right_on='id', how="left")
df = df.merge(df5, left_on="user_id", right_on='id', how="left")
df = df.merge(df6, left_on="user_id", right_on='id', how="left")

df_test = df_test.merge(df_test1, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test2, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test3, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test4, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test5, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test6, left_on="user_id", right_on='id', how="left")

df.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,...,id_y,pred_2,id_x,pred_3,id_y,pred_4,id_x,pred_5,id_y,pred_6
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,...,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000734,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000458,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.004463,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.008806,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000210
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.620179,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.627454,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.607547,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.879773,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.632568
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,...,00001654a9d9f96303d9969d0a4a851714a4bb57,0.148125,00001654a9d9f96303d9969d0a4a851714a4bb57,0.136944,00001654a9d9f96303d9969d0a4a851714a4bb57,0.132337,00001654a9d9f96303d9969d0a4a851714a4bb57,0.305990,00001654a9d9f96303d9969d0a4a851714a4bb57,0.146315
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,...,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000508,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000498,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.004372,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.005401,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.001037
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,...,000028d9e13a595abe061f9b58f3d76ab907850f,0.025337,000028d9e13a595abe061f9b58f3d76ab907850f,0.021932,000028d9e13a595abe061f9b58f3d76ab907850f,0.021137,000028d9e13a595abe061f9b58f3d76ab907850f,0.116135,000028d9e13a595abe061f9b58f3d76ab907850f,0.023210


In [4]:
def empty_fit_params_builder(xvalid, yvalid):
    return {}

def gb_fit_params_builder(xvalid, yvalid):
    return {
        'early_stopping_rounds': 300, 
        'eval_set': [(xvalid, yvalid)], 
        'verbose': 1000
    }

params = [
    (GradientBoostingClassifier, {
        'n_estimators': 100,
        'random_state': 42, 
        'verbose': 1,
        'max_features': 0.1,
    }, 
     empty_fit_params_builder),
    (RandomForestClassifier, {
        'max_depth': 10,
        'max_features': 'auto',
        'class_weight': None,
        'n_estimators': 100,
        'n_jobs': -1,
        'random_state': 42,
        'verbose': 1,
    }, 
     empty_fit_params_builder),
    (XGBClassifier, {
        'learning_rate': 0.012279294564643916,
        'reg_lambda': 3.5341083287733105e-06,
        'reg_alpha': 2.3190508055586554e-05,
        'subsample': 0.9271952162018878,
        'colsample_bytree': 0.9955235948592415,
        'max_depth': 4,
        'random_state': 1,
        'n_estimators': 7000,
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'predictor': "gpu_predictor",
    }, 
     gb_fit_params_builder),
    (LGBMClassifier, {
        'learning_rate': 0.03924735805719372,
        'reg_lambda': 2.0139848865528562,
        'reg_alpha': 2.380837889033024,
        'subsample': 0.6977006896659559,
        'colsample_bytree': 0.9408555739613342,
        'max_depth': 7,
        'random_state': 42,
        'n_estimators': 7000,
    },
     gb_fit_params_builder),
    (CatBoostClassifier, {
        'objective': 'Logloss',
        'colsample_bylevel': 0.0805123693592737,
        'depth': 10,
        'boosting_type': 'Ordered',
        'bootstrap_type': 'MVS',
        'random_state': 1,
    },
     gb_fit_params_builder),
]

In [8]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5", "pred_6"]
df_test = df_test[useful_features]

def build_levels(classifier, params, fit_params_buidler, num):
    sample_submission = pd.read_csv("./data/SampleSubmission.csv")
    
    final_test_predictions = []
    final_valid_predictions = {}
    scores = []
    for fold in range(5):
        xtrain =  df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)
        xtest = df_test.copy()

        valid_ids = xvalid.user_id.values.tolist()

        ytrain = xtrain['CHURN']
        yvalid = xvalid['CHURN']

        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]

        model = classifier(**params)
        
        fit_params = fit_params_buidler(xvalid, yvalid)
        model.fit(xtrain, ytrain, **fit_params)
        preds_valid = model.predict_proba(xvalid)[:, 1]
        test_preds = model.predict_proba(xtest)[:, 1]
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        score = roc_auc_score(yvalid, preds_valid)
        print(fold, score)
        scores.append(score)

    print(np.mean(scores), np.std(scores))
    final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
    final_valid_predictions.columns = ["id", f"pred_{num}"]
    final_valid_predictions.to_csv(f"level1_train_pred_{num}.csv", index=False)

    sample_submission[f'pred_{num}'] = np.mean(np.column_stack(final_test_predictions), axis=1)
    sample_submission[['user_id', f'pred_{num}']].to_csv(f"level1_test_pred_{num}.csv", index=False)    
#     print(sample_submission)
#     print(final_test_predictions)

In [9]:
for i, p in enumerate(tqdm(params)):
    build_levels(p[0], p[1], p[2], i)

  0%|          | 0/5 [00:00<?, ?it/s]

      Iter       Train Loss   Remaining Time 
         1           0.8780            2.33m
         2           0.8152            2.23m
         3           0.7667            2.13m
         4           0.7277            2.11m
         5           0.6957            2.05m
         6           0.6692            2.04m
         7           0.6468            2.01m
         8           0.6279            2.01m
         9           0.6116            1.98m
        10           0.5977            1.94m
        20           0.5290            1.71m
        30           0.5110            1.47m
        40           0.5056            1.25m
        50           0.5037            1.04m
        60           0.5031           49.99s
        70           0.5028           37.57s
        80           0.5027           24.97s
        90           0.5025           12.46s
       100           0.5025            0.00s
0 0.9312996708217351
      Iter       Train Loss   Remaining Time 
         1           0.8783     

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.7s finished


0 0.9314975231809958


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.7s finished


...

[13:51:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68493
[1000]	validation_0-logloss:0.25154
[1025]	validation_0-logloss:0.25154
2 0.931176920088021


h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:51:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68493
[1000]	validation_0-logloss:0.25058
[1725]	validation_0-logloss:0.25057
3 0.9318335231020916


h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:51:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68494
[1000]	validation_0-logloss:0.25226
[1871]	validation_0-logloss:0.25225
4 0.9306976967753648
0.9314750479726464 0.0005043274030113346
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[207]	valid_0's binary_logloss: 0.251535
0 0.9315172641685638
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[340]	valid_0's binary_logloss: 0.250469
1 0.9321714247368614
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[202]	valid_0's binary_logloss: 0.25158
2 0.9311614590070634
Training until validation scores don't improve 

In [17]:
sample_submission = pd.read_csv("./data/SampleSubmission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain['CHURN']
    yvalid = xvalid['CHURN']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    

    params = {
        'random_state': 1, 
        'booster': 'gbtree',
        'n_estimators': 7000,
        'learning_rate': 0.03,
        'max_depth': 2
    }
    
    model = XGBClassifier(
        n_jobs=4,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:, 1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    score = roc_auc_score(yvalid, preds_valid)
    print(fold, score)
    scores.append(score)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv", index=False)

h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:26:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.67329
[711]	validation_0-logloss:0.25154
0 0.9314920060542602


h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:32:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.67332
[685]	validation_0-logloss:0.25050
1 0.9321122137693799


h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:43:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.67329
[630]	validation_0-logloss:0.25059
3 0.9317941847662046


h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:49:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.67335
[689]	validation_0-logloss:0.25223
4 0.9306793422042942
0.9314481254938587 0.0004970086748043533


C:\Users\andre\AppData\Local\Temp/ipykernel_27632/2224334189.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)


In [21]:
sample_submission = pd.read_csv("./data/SampleSubmission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain['CHURN']
    yvalid = xvalid['CHURN']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=3, random_state=1)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:, 1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    score = roc_auc_score(yvalid, preds_valid)
    print(fold, score)
    scores.append(score)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv", index=False)

0 0.9293239254699975
1 0.9299844859833515
2 0.9290454299254474
3 0.9298181691816783
4 0.928237137661946
0.9292818296444842 0.0006213572649891135


C:\Users\andre\AppData\Local\Temp/ipykernel_27632/3169328845.py:36: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)


In [25]:
sample_submission = pd.read_csv("./data/SampleSubmission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain['CHURN']
    yvalid = xvalid['CHURN']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = GradientBoostingClassifier(n_estimators=500, max_depth=3, random_state=10, verbose=1, max_features=0.5)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:, 1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    score = roc_auc_score(yvalid, preds_valid)
    print(fold, score)
    scores.append(score)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("level1_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("level1_test_pred_3.csv", index=False)

      Iter       Train Loss   Remaining Time 
         1           0.8777           19.13m
         2           0.8148           18.63m
         3           0.7660           19.08m
         4           0.7270           18.72m
         5           0.6949           18.41m
         6           0.6682           18.37m
         7           0.6457           18.53m
         8           0.6268           18.65m
         9           0.6106           18.81m
        10           0.5967           18.64m
        20           0.5282           17.92m
        30           0.5101           17.73m
        40           0.5048           17.29m
        50           0.5032           16.69m
        60           0.5026           16.22m
        70           0.5024           15.83m
        80           0.5023           15.36m
        90           0.5022           14.93m
       100           0.5022           14.61m
       200           0.5017           11.15m
       300           0.5013            7.77m
       40

C:\Users\andre\AppData\Local\Temp/ipykernel_27632/770111192.py:36: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)


In [10]:
df = pd.read_csv("./data/train_folds.zip")
df_test = pd.read_csv("./data/Test.zip")
sample_submission = pd.read_csv("./data/SampleSubmission.csv")

df0 = pd.read_csv("level1_train_pred_0.csv")
df1 = pd.read_csv("level1_train_pred_1.csv")
df2 = pd.read_csv("level1_train_pred_2.csv")
df3 = pd.read_csv("level1_train_pred_3.csv")
df4 = pd.read_csv("level1_train_pred_4.csv")

df_test0 = pd.read_csv("level1_test_pred_0.csv")
df_test1 = pd.read_csv("level1_test_pred_1.csv")
df_test2 = pd.read_csv("level1_test_pred_2.csv")
df_test3 = pd.read_csv("level1_test_pred_3.csv")
df_test4 = pd.read_csv("level1_test_pred_4.csv")

df = df.merge(df0, left_on='user_id', right_on="id", how="left")
df = df.merge(df1, left_on='user_id', right_on="id", how="left")
df = df.merge(df2, left_on='user_id', right_on="id", how="left")
df = df.merge(df3, left_on='user_id', right_on="id", how="left")
df = df.merge(df4, left_on='user_id', right_on="id", how="left")

df_test = df_test.merge(df_test0, left_on='user_id', right_on="user_id", how="left")
df_test = df_test.merge(df_test1, left_on='user_id', right_on="user_id", how="left")
df_test = df_test.merge(df_test2, left_on='user_id', right_on="user_id", how="left")
df_test = df_test.merge(df_test3, left_on='user_id', right_on="user_id", how="left")
df_test = df_test.merge(df_test4, left_on='user_id', right_on="user_id", how="left")

df.head()

h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\pandas\core\frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,...,id_x,pred_0,id_y,pred_1,id_x,pred_2,id_y,pred_3,id,pred_4
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,...,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000979,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000370,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000563,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000519,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000889
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.623912,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.618955,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.622513,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.624550,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.664610
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,...,00001654a9d9f96303d9969d0a4a851714a4bb57,0.139086,00001654a9d9f96303d9969d0a4a851714a4bb57,0.136379,00001654a9d9f96303d9969d0a4a851714a4bb57,0.139097,00001654a9d9f96303d9969d0a4a851714a4bb57,0.137898,00001654a9d9f96303d9969d0a4a851714a4bb57,0.145545
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,...,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000941,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000289,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000264,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000191,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000566
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,...,000028d9e13a595abe061f9b58f3d76ab907850f,0.019892,000028d9e13a595abe061f9b58f3d76ab907850f,0.022725,000028d9e13a595abe061f9b58f3d76ab907850f,0.023450,000028d9e13a595abe061f9b58f3d76ab907850f,0.025753,000028d9e13a595abe061f9b58f3d76ab907850f,0.020862


In [11]:
df_test.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,...,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,pred_0,pred_1,pred_2,pred_3,pred_4
0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,THIES,K > 24 month,5000.0,5.0,5000.0,1667.0,5.0,NaN,378.0,...,NaN,NO,42,On-net 1000F=10MilF;10d,5.0,0.001155,0.001115,0.001170,0.001086,0.001086
1,000055d41c8a62052dd426592e8a4a3342bf565d,NaN,I 18-21 month,300.0,2.0,326.0,109.0,3.0,397.0,NaN,...,NaN,NO,41,"Data: 100 F=40MB,24H",1.0,0.068719,0.071787,0.072642,0.071480,0.071362
2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,DAKAR,K > 24 month,3300.0,25.0,3400.0,1133.0,26.0,7150.0,0.0,...,NaN,NO,57,"Data: 100 F=40MB,24H",22.0,0.001132,0.000461,0.000259,0.000388,0.000732
3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NO,9,NaN,NaN,0.401654,0.395942,0.398299,0.397624,0.396494
4,0000bae5480628cf8fe51ad84bcb39772fc79224,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NO,10,NaN,NaN,0.376721,0.374178,0.375060,0.374096,0.378229


In [12]:
useful_features = ["pred_0", "pred_1", "pred_2", "pred_3", 'pred_4']
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain['CHURN']
    yvalid = xvalid['CHURN']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LogisticRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:, 1]
    final_predictions.append(test_preds)
    score = roc_auc_score(yvalid, preds_valid)
    print(fold, score)
    scores.append(score)

print(np.mean(scores), np.std(scores))
# 0 0.931452296769149
# 1 0.9320405785857452
# 2 0.9311174885821525
# 3 0.9316499975730037
# 4 0.9306373918079478
# 0.9313795506635996 0.0004764174098602453

0 0.9314145083291796
1 0.9319840853531944
2 0.9311070992866594
3 0.931637670244932
4 0.9306176693234288
0.9313522065074787 0.0004656746423203319


In [13]:
sample_submission = pd.read_csv("./data/SampleSubmission.csv")
sample_submission['CHURN'] = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("./data/submission-stack-3.csv", index=False)

sample_submission

,user_id,CHURN
0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,0.027271
1,000055d41c8a62052dd426592e8a4a3342bf565d,0.044138
2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,0.027071
3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,0.298354
4,0000bae5480628cf8fe51ad84bcb39772fc79224,0.266843
...,...,...
380122,fffe7e03c7eede2ad0a728ee516c4d342dd16107,0.027125
380123,fffec230e6a1aa51ab37d0051ece42de611e71c6,0.859351
380124,ffff0dcc1ab9812bf205b6d76e9d084053cd96f5,0.134142
380125,ffff91ea6a09a0c8ea42bc6ae33df4b5e06283dc,0.057445
